In [8]:
import librosa
#import librosa.display
import numpy as np
#import matplotlib.pyplot as plt
import tensorflow as tf
#from matplotlib.pyplot import specgram
import pandas as pd
import os
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

In [9]:
angryFiles = os.listdir('emotion_train_set/angry')

In [10]:
angryFiles[:5]

['379_4.wav', '441_3.json', '845_2.wav', '1986_5.wav', '210_3.json']

In [11]:
disgustFiles = os.listdir('emotion_train_set/disgust')

In [12]:
fearFiles = os.listdir('emotion_train_set/fear')

In [13]:
happyFiles = os.listdir('emotion_train_set/happy')

In [14]:
neutralFiles = os.listdir('emotion_train_set/neutral')

In [15]:
sadFiles = os.listdir('emotion_train_set/sad')

In [16]:
surpriseFiles = os.listdir('emotion_train_set/surprise')

In [17]:
angryFiles.sort()
disgustFiles.sort()
fearFiles.sort()
happyFiles.sort()
neutralFiles.sort()
sadFiles.sort()
surpriseFiles.sort()

In [18]:
import json

In [19]:
angryJSONFiles = os.listdir('emotion_train_set/angry')
angryJSONFiles.sort()
angryData=[]
for file in angryJSONFiles:
    try:
        with open('emotion_train_set/angry/'+file) as json_data:
            d = json.load(json_data)
            angryData.append(d['features'])
    except: pass

In [20]:
disgustJSONFiles = os.listdir('emotion_train_set/disgust')
disgustJSONFiles.sort()
disgustData=[]
for file in disgustJSONFiles:
    try:
        with open('emotion_train_set/disgust/'+file) as json_data:
            d = json.load(json_data)
            disgustData.append(d['features'])
    except: pass


In [21]:
fearJSONFiles = os.listdir('emotion_train_set/fear')
fearJSONFiles.sort()
fearData=[]
for file in fearJSONFiles:
    try:
        with open('emotion_train_set/fear/'+file) as json_data:
            d = json.load(json_data)
            fearData.append(d['features'])
    except: pass

In [22]:
happyJSONFiles = os.listdir('emotion_train_set/happy')
happyJSONFiles.sort()
happyData=[]
for file in happyJSONFiles:
    try:
        with open('emotion_train_set/happy/'+file) as json_data:
            d = json.load(json_data)
            happyData.append(d['features'])
    except: pass

In [23]:
neutralJSONFiles = os.listdir('emotion_train_set/neutral')
neutralJSONFiles.sort()
neutralData=[]
for file in neutralJSONFiles:
    try:
        with open('emotion_train_set/neutral/'+file) as json_data:
            d = json.load(json_data)
            neutralData.append(d['features'])
    except: pass

In [24]:
sadJSONFiles = os.listdir('emotion_train_set/sad')
sadJSONFiles.sort()
sadData=[]
for file in sadJSONFiles:
    try:
        with open('emotion_train_set/sad/'+file) as json_data:
            d = json.load(json_data)
            sadData.append(d['features'])
    except: pass

In [25]:
surpriseJSONFiles = os.listdir('emotion_train_set/surprise')
surpriseJSONFiles.sort()
surpriseData=[]
for file in surpriseJSONFiles:
    try:
        with open('emotion_train_set/surprise/'+file) as json_data:
            d = json.load(json_data)
            surpriseData.append(d['features'])
    except: pass

In [26]:
angryDf = pd.DataFrame(angryData)
disgustDf = pd.DataFrame(disgustData)
fearDf = pd.DataFrame(fearData)
happyDf = pd.DataFrame(happyData)
neutralDf = pd.DataFrame(neutralData)
sadDf = pd.DataFrame(sadData)
surpriseDf = pd.DataFrame(surpriseData)

In [27]:
angryDf['label'] = 'angry'
disgustDf['label'] = 'disgust'
fearDf['label'] = 'fear'
happyDf['label'] = 'happy'
neutralDf['label'] = 'neutral'
sadDf['label'] = 'sad'
surpriseDf['label'] = 'surprise'

In [28]:
emotionsDf = pd.concat([angryDf, disgustDf, fearDf, happyDf, neutralDf, sadDf, surpriseDf])

In [29]:
from sklearn.utils import shuffle
shuffledEmotionsDf = shuffle(emotionsDf)

In [30]:
divider = np.random.rand(len(shuffledEmotionsDf)) < 0.85
train = shuffledEmotionsDf[divider]
test = shuffledEmotionsDf[~divider]

In [31]:
trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]

In [32]:
testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]

In [33]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

/Users/lucaslyon/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
X_train.shape

(5707, 272)

In [35]:
model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(272,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(7))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 272, 128)          768       
_________________________________________________________________
activation_1 (Activation)    (None, 272, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 272, 128)          82048     
_________________________________________________________________
activation_2 (Activation)    (None, 272, 128)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 272, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 34, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 34, 128)           82048     
__________

In [37]:
print('Pad sequences')
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

Pad sequences


In [38]:
from keras.losses import mean_squared_error
model.compile(loss=mean_squared_error, optimizer=opt,metrics=['accuracy'])

In [ ]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=32, epochs=100, validation_data=(x_testcnn, y_test))

Train on 5707 samples, validate on 1013 samples
Epoch 1/100
5707/5707 [==============================] - 25s 4ms/step - loss: 0.1375 - acc: 0.2159 - val_loss: 0.1147 - val_acc: 0.2646
Epoch 2/100
5707/5707 [==============================] - 25s 4ms/step - loss: 0.1248 - acc: 0.2308 - val_loss: 0.1139 - val_acc: 0.2932
Epoch 3/100
5707/5707 [==============================] - 25s 4ms/step - loss: 0.1200 - acc: 0.2457 - val_loss: 0.1126 - val_acc: 0.3119
Epoch 4/100
5707/5707 [==============================] - 30s 5ms/step - loss: 0.1175 - acc: 0.2581 - val_loss: 0.1121 - val_acc: 0.3021
Epoch 5/100
5707/5707 [==============================] - 26s 5ms/step - loss: 0.1164 - acc: 0.2592 - val_loss: 0.1120 - val_acc: 0.3011
Epoch 6/100
5707/5707 [==============================] - 27s 5ms/step - loss: 0.1157 - acc: 0.2637 - val_loss: 0.1117 - val_acc: 0.3119
Epoch 7/100
5707/5707 [==============================] - 27s 5ms/step - loss: 0.1143 - acc: 0.2903 - val_loss: 0.1125 - val_acc: 0.2922


5707/5707 [==============================] - 27s 5ms/step - loss: 0.1079 - acc: 0.3529 - val_loss: 0.1091 - val_acc: 0.3386
Epoch 61/100
5707/5707 [==============================] - 28s 5ms/step - loss: 0.1078 - acc: 0.3524 - val_loss: 0.1088 - val_acc: 0.3416
Epoch 62/100
5707/5707 [==============================] - 30s 5ms/step - loss: 0.1076 - acc: 0.3564 - val_loss: 0.1090 - val_acc: 0.3327
Epoch 63/100
5707/5707 [==============================] - 27s 5ms/step - loss: 0.1076 - acc: 0.3538 - val_loss: 0.1090 - val_acc: 0.3544
Epoch 64/100
5707/5707 [==============================] - 26s 5ms/step - loss: 0.1079 - acc: 0.3601 - val_loss: 0.1086 - val_acc: 0.3544
Epoch 65/100
5707/5707 [==============================] - 26s 5ms/step - loss: 0.1077 - acc: 0.3582 - val_loss: 0.1085 - val_acc: 0.3495
Epoch 66/100
5707/5707 [==============================] - 35s 6ms/step - loss: 0.1077 - acc: 0.3554 - val_loss: 0.1086 - val_acc: 0.3475
Epoch 67/100
5707/5707 [==============================

In [ ]:
score = model.evaluate(x_testcnn, y_test, verbose=0)
print("test loss:", score[0])
print("test accuracy:", score[1])

In [ ]:
from keras.models import load_model
model.save('full_model.h5') #saving
#model = load_model('full_model.h5') #loading